<a href="https://colab.research.google.com/github/annakalinina18/star-fle/blob/main/annotation_with_LLM/qwen3max_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import random
import re
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# =========================
# 0. CLIENT (Qwen via DashScope OpenAI-compatible API)
# =========================
# export DASHSCOPE_API_KEY="..."
api_key = ""

client = OpenAI(
    api_key=api_key,
    base_url="https://dashscope-intl.aliyuncs.com/compatible-mode/v1",
    # si besoin (région Chine) :
    # base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# =========================
# 1. MODÈLE
# =========================
MODEL_NAME = "qwen3-max"

# =========================
# 2. RETRY / THROTTLE
# =========================
MAX_RETRIES = 6
MAX_BACKOFF_SEC = 25.0
THROTTLE_PER_CALL_SEC = 0.10

def _is_retryable_error(e: Exception) -> bool:
    msg = str(e).lower()
    return (
        "429" in msg or "rate" in msg or
        "503" in msg or "unavailable" in msg or
        "overload" in msg or "timeout" in msg or
        "server" in msg
    )

def chat_completion_with_retry(messages, temperature=0, max_tokens=80):
    last_exc = None
    for attempt in range(MAX_RETRIES):
        try:
            resp = client.chat.completions.create(
                model=MODEL_NAME,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            time.sleep(THROTTLE_PER_CALL_SEC)
            return resp
        except Exception as e:
            last_exc = e
            if _is_retryable_error(e):
                wait = min(MAX_BACKOFF_SEC, (2 ** attempt) + random.uniform(0, 1))
                print(f"⚠️ API retryable error — retry in {wait:.1f}s ({attempt+1}/{MAX_RETRIES})")
                time.sleep(wait)
                continue
            raise
    raise RuntimeError(f"Échec après retries. Dernière erreur: {last_exc}")

# =========================
# 3. BASELINE PROMPT (4 définitions générales)
# =========================
BASELINE_PROMPT = """
Tu es linguiste.

Ta tâche est de classer une expression nominale française
dans UNE SEULE des catégories suivantes, en te fondant sur
une compréhension linguistique générale (pas de guide spécifique).

Catégories :

1) Expression_idiomatique
Expression figée dont le sens global n’est pas directement déductible
du sens littéral de ses mots.
Exemples : « lune de miel », « forêt noire » (gâteau)

2) Collocation_opaque
Association de mots relativement conventionnelle,
dont le sens implique une image, une métaphore ou une métonymie.
Exemples : « fil rouge », « train de vie »

3) Collocation_transparente
Association de mots fréquente ou conventionnelle,
dont le sens est globalement déductible des mots qui la composent.
Exemples : « événement culturel », « roman policier »

4) Expression_libre
Combinaison de mots construite librement en discours,
sans caractère figé ou conventionnel particulier.
Exemples : « livre intéressant », « maison ancienne »

Contraintes :
- Choisis UNE seule catégorie.
- Réponds uniquement selon le format ci-dessous.

FORMAT OBLIGATOIRE :
Catégorie : <Expression_idiomatique | Collocation_opaque | Collocation_transparente | Expression_libre>
Explication : <une phrase très courte>

Expression : {expression}
Contexte : {contexte}
"""

ALLOWED = {
    "Expression_idiomatique",
    "Collocation_opaque",
    "Collocation_transparente",
    "Expression_libre",
}

def extract_category(text: str) -> str:
    if not text:
        return "INVALID"
    m = re.search(r"(?im)^\s*catégorie\s*:\s*(.+?)\s*$", text)
    if not m:
        return "INVALID"
    cat = m.group(1).strip()
    return cat if cat in ALLOWED else "INVALID"

# =========================
# 4. 1 appel / expression
# =========================
def classify_baseline(expression, examples):
    contexte = "" if examples is None or pd.isna(examples) else str(examples)
    prompt = BASELINE_PROMPT.format(
        expression=str(expression).strip(),
        contexte=contexte
    )

    resp = chat_completion_with_retry(
        messages=[{"role": "user", "content": prompt}],
        temperature=0,
        max_tokens=80,
    )

    raw = (resp.choices[0].message.content or "").strip()
    cat = extract_category(raw)
    return cat, raw

# =========================
# 5. TRAITEMENT EXCEL
# =========================
input_file = "nominal_part_7.xlsx"
df = pd.read_excel(input_file)

df["llm_category"] = None
df["llm_raw_response"] = None

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Baseline (Qwen3-Max)"):
    expr = row.get("expression")
    ex = row.get("examples_joined")

    if pd.isna(expr) or not str(expr).strip():
        df.at[idx, "llm_category"] = "N/A"
        df.at[idx, "llm_raw_response"] = "N/A"
        continue

    cat, raw = classify_baseline(expr, ex)
    df.at[idx, "llm_category"] = cat
    df.at[idx, "llm_raw_response"] = raw

output_file = "annotated_nominal_part_7_qwen3_max_baseline_4defs.xlsx"
df.to_excel(output_file, index=False)

print(f"Saved: {output_file}")


Baseline (Qwen3-Max): 100%|██████████| 100/100 [03:54<00:00,  2.35s/it]

Saved: annotated_nominal_part_7_qwen3_max_baseline_4defs.xlsx
